###1. 데이터 입수

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/dacon')

In [3]:
import pandas as pd
import numpy as np

test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")

###2. 탐색적 데이터 분석

 먼저 목표가 되는 credit 의 분포를 봅시다. 

In [8]:
train_df.groupby("credit")["index"].count()

credit
0.0     3222
1.0     6267
2.0    16968
Name: index, dtype: int64

숫자형 변수 간의 상관관계를 한번 살펴보면

In [9]:
cormat = train_df.corr()
cormat

,index,child_num,income_total,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,family_size,begin_month,credit
index,1.000000,0.001012,0.009848,-0.008901,0.004259,NaN,-0.005575,-0.001790,0.005332,-0.002867,0.002181,-0.007841
child_num,0.001012,1.000000,0.032186,0.332816,-0.228159,NaN,0.051521,-0.010555,0.016120,0.890530,-0.007229,0.004081
income_total,0.009848,0.032186,1.000000,0.064155,-0.166478,NaN,-0.034207,0.019013,0.089882,0.023839,-0.018047,0.008555
DAYS_BIRTH,-0.008901,0.332816,0.064155,1.000000,-0.616776,NaN,0.178752,-0.029247,0.111238,0.298108,0.057306,-0.025187
DAYS_EMPLOYED,0.004259,-0.228159,-0.166478,-0.616776,1.000000,NaN,-0.241827,-0.007749,-0.091090,-0.223077,0.007964,-0.003798
FLAG_MOBIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
work_phone,-0.005575,0.051521,-0.034207,0.178752,-0.241827,NaN,1.000000,0.310256,-0.031048,0.068705,-0.008563,-0.003134
phone,-0.001790,-0.010555,0.019013,-0.029247,-0.007749,NaN,0.310256,1.000000,0.012494,-0.000132,-0.014286,0.003452
email,0.005332,0.016120,0.089882,0.111238,-0.091090,NaN,-0.031048,0.012494,1.000000,0.016122,0.002320,0.014812
family_size,-0.002867,0.890530,0.023839,0.298108,-0.223077,NaN,0.068705,-0.000132,0.016122,1.000000,-0.023032,0.008227


숫자의 상관관계를 보면 숫자만으로 해결하기는 힘들꺼란 생각이 듭니다

###3. 데이터 전처리

먼저 카테고리형 변수와 숫자형 변수를 해결해야 합니다

In [10]:
train_obj_df = train_df.select_dtypes(include='object')	# 카테고리형
train_num_df = train_df.select_dtypes(exclude='object') # 숫자형

test_obj_df = test_df.select_dtypes(include='object')	# 카테고리형
test_num_df = test_df.select_dtypes(exclude='object') # 숫자형

get_dummies 함수를 이용하면 문자형으로 (혹은 일부 숫자)를 되어 있는 범주형 변수들을 각 요소의 유무 판별 컬럼으로 분해해줄 수 있습니다. drop_first = True를 하면 첫번째 컬럼을 생략할 수 있게 되는데, 이러면 첫번째 요소에 해당하는 데이터는 관련 칼럼이 모두 0이 되겠죠. 예를 들어 범주가 네가지이면 1번 범주는 000, 2번 범주는 100, 3번범주는 010, 4번 범주는 001 이런식으로 구분할 수 있습니다. 이런걸 One-Hot encoding이라고 합니다.

In [12]:
train_dummy_df = pd.get_dummies(train_obj_df, drop_first=True)
train_dummy_df.index = train_df.index


test_dummy_df = pd.get_dummies(test_obj_df, drop_first=True)
test_dummy_df.index = test_df.index

train_dummy_df.head()

,gender_M,car_Y,reality_Y,income_type_Pensioner,income_type_State servant,income_type_Student,income_type_Working,edu_type_Higher education,edu_type_Incomplete higher,edu_type_Lower secondary,edu_type_Secondary / secondary special,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,occyp_type_Cleaning staff,occyp_type_Cooking staff,occyp_type_Core staff,occyp_type_Drivers,occyp_type_HR staff,occyp_type_High skill tech staff,occyp_type_IT staff,occyp_type_Laborers,occyp_type_Low-skill Laborers,occyp_type_Managers,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff
0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,1,1,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


범주형과 숫자형을 다시 합친 후

In [14]:
train_aft_df = pd.concat( [train_num_df, train_dummy_df],  axis = 1 )
train_aft_df = train_aft_df.drop(columns = ["index"])
test_aft_df = pd.concat( [test_num_df, test_dummy_df],  axis = 1 )
test_aft_df = test_aft_df.drop(columns = ["index"])

목표치인 credit을 떼어내어 y로, 나머지는 x로 지정합니다

In [15]:
train_y_df  = train_aft_df["credit"] 
train_x_df = train_aft_df.drop(columns = ["credit"])

먼저 train data의 일부를 train, 나머지를 test용으로 지정해봅시다. train_index를 중심으로 split 할껍니다. 보통 70% 정도로 학습하고 나머지 30%로 확인해보더군요.

In [16]:
train_index = int(len(train_y_df) * 0.7)

In [17]:
train_train_y_df = train_y_df[:train_index]
train_test_y_df = train_y_df[train_index:]
train_train_x_df = train_x_df[:train_index]
train_test_x_df = train_x_df[train_index:]


###4. 모델 적용

그룹을 나누기 위해서 제일 적절해 보이는 기법은 KNN이길래 먼저 이걸로 해봅니다.

In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)

In [19]:
knn.fit(train_train_x_df, train_train_y_df)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

dacon에서 득점을 체크하는 방법은 보통 log_loss인데, sklearn에 패키지가 있고, predict가 아닌 predict_proba 함수로 결과값을 구하면 각 단계에 대한 확률로 표시되기 때문에 이걸로 계산해보면 됩니다. 예를 들면 이 경우에 train_pred_proba에는 [0.1,0.01,0.8] 같은 식으로 각 클래스의 확률을 적습니다

In [20]:
from sklearn.metrics import log_loss

train_pred_proba = knn.predict_proba(train_test_x_df)
log_loss(train_test_y_df, train_pred_proba)



12.683362709897283

이게 처음 돌렸던 값인데... 이 정도면 보통 꼴찌가 나옵니다. n_neighbor 값을 바꿔봅시다.

In [42]:
knn = KNeighborsClassifier(n_neighbors=170, weights= "uniform", metric = "manhattan")
knn.fit(train_train_x_df, train_train_y_df)
train_pred_proba = knn.predict_proba(train_test_x_df)
log_loss(train_test_y_df, train_pred_proba)

0.8837954377495592

숫자를 높여보니 값이 훨씬 개선 되어 나왔습니다. 그러면 적절한 파라메터를 찾기 위해 gridsearchCV를 쓸 수 있지 않을까요?

In [26]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_neighbors' : [3,5,11,19,31,50,90,110],
    'weights' : ['uniform','distance'],
    'metric' :['euclidean','manhattan'],
}

gs = GridSearchCV(KNeighborsClassifier(), 
                  grid_params,
                  verbose = 1,
                  cv = 3,
                  n_jobs = -1
                  )

gs_result = gs.fit(train_train_x_df, train_train_y_df)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:   52.4s finished


In [27]:
gs_result.best_score_
gs_result.best_estimator_
gs_result.best_params_

{'metric': 'euclidean', 'n_neighbors': 110, 'weights': 'distance'}

In [30]:
grid_params = {
    'n_neighbors' : range(100,1000, 100 ),
    'weights' : ['uniform','distance'],
    'metric' :['euclidean','manhattan'],
}

gs = GridSearchCV(KNeighborsClassifier(), 
                  grid_params,
                  verbose = 1,
                  cv = 3,
                  n_jobs = -1
                  )

gs_result = gs.fit(train_train_x_df, train_train_y_df)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  3.0min finished


In [31]:
gs_result.best_score_
gs_result.best_estimator_
gs_result.best_params_

{'metric': 'manhattan', 'n_neighbors': 400, 'weights': 'distance'}

In [35]:
train_pred_proba = gs_result.predict_proba(train_test_x_df)
log_loss(train_test_y_df, train_pred_proba)

2.0126923023979884

아니 이게 왠일인가요... 이게 바로 과적합인가요

In [49]:
grid_params = {
    'n_neighbors' : range(100,120, 1 ),
    'weights' : ['uniform'],
    'metric' :['manhattan'],
}

gs = GridSearchCV(KNeighborsClassifier(), 
                  grid_params,
                  verbose = 1,
                  cv = 3,
                  n_jobs = -1
                  )

gs_result = gs.fit(train_train_x_df, train_train_y_df)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   37.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   49.1s finished


weights 를 distance로 하면 과적합이 나오는 모양이라 uniform으로 맞추고, 다른걸 바꿔서 해봅니다. 그러나 n_neighbors는 100 전후에서 비슷하게 나오네요.

In [50]:
gs_result.best_score_
gs_result.best_estimator_
gs_result.best_params_

{'metric': 'manhattan', 'n_neighbors': 102, 'weights': 'uniform'}

In [51]:
train_pred_proba = gs_result.predict_proba(train_test_x_df)
log_loss(train_test_y_df, train_pred_proba)

0.8827402693829497